In [1]:
import pandas as pd
import numpy as np

In [25]:
dict_obj = {'key1' : ['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      'two', 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8)}
df_obj = pd.DataFrame(dict_obj)
print(df_obj)

   data1  data2 key1   key2
0      1      5    a    one
1      9      7    b    one
2      9      4    a    two
3      2      1    b  three
4      5      6    a    two
5      4      1    b    two
6      9      2    a    one
7      3      6    a  three


In [9]:
# 按key2来分组，进行sum() 运算，并将计算后的data1 和 data2的结果添加列名前缀 sum_
df_sum = df_obj.groupby("key2").sum().add_prefix("sum_")
print(df_sum)

       sum_data1  sum_data2
key2                       
one            5         25
three          6         11
two           19          8


### merge 进行表合并

In [ ]:
# 主键：主键是一张表的主键，必须是唯一且非空的。
# 外键：是另一张表中，与这张表的某个字段的类型、字段名都是相同的字段。
# 一般用来关联两张或多张表的数据表用。

In [14]:
# left_on：左边数据表的"外键"
# right_on：右边数据表的"外键"
# left_index：使用左边的DataFrame对象的行索引做为连接健
# right_index：使用右边的DataFrame对象的行索引做为连接健
# 一般情况下，外键和连接健的方向是相反的。

#　第一种方法，merge可以将两张表合并，但是前提条件是 必须有相同的外键
key1_merge = pd.merge(df_obj, df_sum, left_on = "key2", right_index = True)
print(key1_merge)

   data1  data2 key1   key2  sum_data1  sum_data2
0      1      9    a    one          5         25
1      3      7    b    one          5         25
6      1      9    a    one          5         25
2      5      2    a    two         19          8
4      9      4    a    two         19          8
5      5      2    b    two         19          8
3      1      9    b  three          6         11
7      5      2    a  three          6         11


## transform 表合并

In [15]:
print(df_obj)
print(df_sum)

   data1  data2 key1   key2
0      1      9    a    one
1      3      7    b    one
2      5      2    a    two
3      1      9    b  three
4      9      4    a    two
5      5      2    b    two
6      1      9    a    one
7      5      2    a  three
       sum_data1  sum_data2
key2                       
one            5         25
three          6         11
two           19          8


In [24]:
# 第二种方法，使用transform进行聚合运算， 运算结果和原始数据形状保持一致
key1_trans = df_obj.groupby("key1").transform(sum).add_prefix("sum_")
# 将key1_trans 的列数据添加到原始数据里
df_obj[key1_trans.columns] = key1_trans
print(df_obj)

   data1  data2 key1   key2 sum_data1 sum_data2           sum_key2
0      9      5    a    one        22        22  onetwotwoonethree
1      6      7    b    one        20        16        onethreetwo
2      3      1    a    two        22        22  onetwotwoonethree
3      7      3    b  three        20        16        onethreetwo
4      2      6    a    two        22        22  onetwotwoonethree
5      7      6    b    two        20        16        onethreetwo
6      1      9    a    one        22        22  onetwotwoonethree
7      7      1    a  three        22        22  onetwotwoonethree


In [27]:
# transform可以传入自定义函数
def func(df):
    return df.sum()

newkey1_trans = df_obj.groupby("key1").transform(func).add_prefix("newsum_")
df_obj[newkey1_trans.columns] = newkey1_trans
print(df_obj)

   data1  data2 key1   key2 newsum_data1 newsum_data2        newsum_key2
0      1      5    a    one           27           23  onetwotwoonethree
1      9      7    b    one           15            9        onethreetwo
2      9      4    a    two           27           23  onetwotwoonethree
3      2      1    b  three           15            9        onethreetwo
4      5      6    a    two           27           23  onetwotwoonethree
5      4      1    b    two           15            9        onethreetwo
6      9      2    a    one           27           23  onetwotwoonethree
7      3      6    a  three           27           23  onetwotwoonethree


## apply 

In [33]:

## 案例 starcraft APM
data_path = "starcraft.csv"

# 读取本地csv文件的 指定列
df_data = pd.read_csv(data_path, usecols=['LeagueIndex', 'Age', 'HoursPerWeek', 
                                             'TotalHours', 'APM'])
#print(df_data)

def top_n(df_obj):
    # 返回根据APM排序后的结果的前3位数据
    return df_obj.sort_values(by="APM", ascending=False)[:3]

# 对df_obj按LeagueIndex进行分组，并调用apple(top_n)
sort_df = df_data.groupby("LeagueIndex").apply(top_n)
print(sort_df)

                  LeagueIndex   Age  HoursPerWeek  TotalHours       APM
LeagueIndex                                                            
1           2214            1  20.0          12.0       730.0  172.9530
            2246            1  27.0           8.0       250.0  141.6282
            1753            1  20.0          28.0       100.0  139.6362
2           3062            2  20.0           6.0       100.0  179.6250
            3229            2  16.0          24.0       110.0  156.7380
            1520            2  29.0           6.0       250.0  151.6470
3           1557            3  22.0           6.0       200.0  226.6554
            484             3  19.0          42.0       450.0  220.0692
            2883            3  16.0           8.0       800.0  208.9500
4           2688            4  26.0          24.0       990.0  249.0210
            1759            4  16.0           6.0        75.0  229.9122
            2637            4  23.0          24.0       650.0  2